# Modeling with GNN-ePC-SAFT

Model combining graph neural network with ePC-SAFT


## Starting point

In [1]:
import torch
from torchmetrics import MeanAbsolutePercentageError
from torch.nn import HuberLoss
from torch_geometric.loader import DataLoader
import os.path as osp, pickle
from data.graphdataset import ThermoMLDataset, ramirez, ThermoMLpara
from train.parametrisation import  MAPE
from epcsaft import epcsaft_cython

In [2]:
device = torch.device("cpu")

In [3]:
HLoss = HuberLoss("mean").to(device)
mape = MeanAbsolutePercentageError().to(device)

In [4]:
pcsaft_den = epcsaft_cython.PCSAFT_den.apply
pcsaft_vp = epcsaft_cython.PCSAFT_vp.apply

path = osp.join("./data", "thermoml")
test_loader = ThermoMLDataset(path)

In [16]:
train_loader = ThermoMLpara(path)
para_data = {}
for graph in train_loader:
    para_data[graph.InChI] = [graph.para]

In [17]:
n=0
for gh in test_loader:
    if gh.InChI in para_data:
        n += 1 
print(n, test_loader)

1962 ThermoMLDataset(2197)


In [18]:
def test(para_data):
    data = {"inchis":[],"mden":[],"mvp":[]} 
    for gh in test_loader:
        if gh.InChI not in para_data:
            continue
        parameters = para_data[gh.InChI][0]
        params = parameters.squeeze().to(torch.float64).numpy()
        rho = gh.rho.view(-1, 5).to(torch.float64).numpy()
        vp = gh.vp.view(-1, 5).to(torch.float64).numpy()
        mden, mvp = MAPE(params, rho, vp)
        data["inchis"].append(gh.InChI)
        data["mden"].append(mden)
        data["mvp"].append(mvp)
    return data

In [19]:
tmlpara = test(para_data)

In [22]:
import polars as pl
tmlpara = pl.DataFrame(tmlpara)

In [26]:
tmlpara.write_csv("./reftmlpara.csv")

In [41]:
with open("./data/thermoml/raw/para3_fitted.pkl", "rb") as file:
    fitted_para = pickle.load(file)

In [42]:
data = {"inchis":[],"mden":[],"mvp":[]}
for inchi in fitted_para:
    data['inchis'].append(inchi)
    data['mden'].append(fitted_para[inchi][1])
    data['mvp'].append(fitted_para[inchi][2])

In [43]:
para = pl.DataFrame(data)

In [45]:
para.write_csv('refparametrisation.csv')

In [29]:
train_loader = ramirez('./data/ramirez2022')
para_data = {}
for graph in train_loader:
    para_data[graph.InChI] = [graph.para]

In [30]:
rapara = test(para_data)

In [31]:
rapara = pl.DataFrame(rapara)

In [33]:
rapara.write_csv('reframirez.csv')